# Лабораторная работа №2

## Подготока входных данных

Импортируем необходимые библиотеки.

In [27]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

Загружаем датасет и выбераем из него 50000 тыс. случайных записей.

In [29]:
dataset = pd.read_csv('Australia Rainfall.csv', delimiter=',').sample(50000)

Убераем столбцы, которые не содержат полезной информации, а также столбцы содержащие большое количество пустых значений. После этого убераем все записи, у которых есть хотя бы одно пустое значений.

In [31]:
X = dataset.drop(columns=['Evaporation', 'Sunshine', 'Date', 'Cloud9am', 'Cloud3pm'])
X.dropna(inplace=True)

Заменяем столбцы со строковыми значениями на соответствующие им целочисленные. Столбцы RainToday и RainTomorrow заменяем на bool-аналоги, так как они содержат всего 2 возможных значения.

In [35]:
locations = X['Location'].unique()
X['Location'] = X['Location'].map({value: i for i, value in enumerate(locations)})

X['RainToday'] = X['RainToday'].map({'No': False, 'Yes': True})
X['RainTomorrow'] = X['RainTomorrow'].map({'No': False, 'Yes': True})

wgds = X['WindGustDir'].unique()
X['WindGustDir'] = X['WindGustDir'].map({value: i for i, value in enumerate(wgds)})

wd9s = X['WindDir9am'].unique()
X['WindDir9am'] = X['WindDir9am'].map({value: i for i, value in enumerate(wd9s)})

wd9s = X['WindDir3pm'].unique()
X['WindDir3pm'] = X['WindDir3pm'].map({value: i for i, value in enumerate(wd9s)})

Отделяем таргетный столбец RainTomorrow от остального датасета.

In [39]:
y = X['RainTomorrow']
X = X.drop('RainTomorrow', axis=1)

Нормализуем данные в пределах от 0 до 1.

In [42]:
scaler = MinMaxScaler()
X_ans = scaler.fit_transform(X)

Разделяем датасет на тествую и тренировочную части в отношении 70/30.

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X_ans, y, test_size=0.3)

## Задание 1

*Провести классификацию найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.*

Импортируем класс SVC для классификации датасета методом опорных векторов.

In [50]:
from sklearn.svm import SVC

Создаём объект класса SVC, реализующий метод опорных векторов.

In [53]:
svc_classifier = SVC()

Определяем параметры, которые мы хотим оптимизировать для модели SVM.
*  C: Это параметр регуляризации, который контролирует баланс между точностью на тренировочных данных и способностью модели обобщать на новые данные. Чем меньше значение C, тем больше регуляризация, и модель будет более склонна к обобщению на новые данные, но может быть менее точной на тренировочных данных.
* gamma: Этот параметр определяет радиус влияния ядра. Он контролирует, как близко должны быть точки данных, чтобы быть “похожими” с точки зрения модели. Чем больше значение gamma, тем уже радиус влияния, и модель будет более чувствительна к локальным особенностям данных.
* kernel: Это тип ядра модели SVM. Ядро определяет, как модель вычисляет похожесть между точками данных.

In [56]:
params ={
    'C': [1,0.1,0.01,0.001],
    'gamma': [1,10,100,1000],
    'kernel':['linear','sigmoid','rbf']
}

Создаём объект класса GridSearchCV, выполняющий поиск оптимальных параметров модели, перебирая все возможные комбинации значений параметров.
* c5=5 - определяет количество фолдов (разделений) для кросс-валидации. В данном случае 5-кратная кросс-валидаци .
* 
n_jobs=-1 - определяет количество ядер процессора, которые будут использоваться для параллельного выполнения Grid Search.

In [59]:
svc_grid = GridSearchCV(svc_classifier, params, cv=5, n_jobs=-1)
svc_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(), n_jobs=-1,
             param_grid={'C': [1, 0.1, 0.01, 0.001],
                         'gamma': [1, 10, 100, 1000],
                         'kernel': ['linear', 'sigmoid', 'rbf']})

Обучаем модель на тренировочных данных.

In [61]:
best_model = SVC(**svc_grid.best_params_)
best_model.fit(X_train, y_train)

SVC(C=1, gamma=1, kernel='linear')

Применяем обученную модель к тестовым данным X_test и делаем предсказания.

In [63]:
predicted = best_model.predict(X_test)

Вывод отчёта.

In [67]:
print('Параметры:', svc_grid.best_params_)
print('Оценка:\n', metrics.classification_report(y_test, predicted,digits=5))

Параметры: {'C': 1, 'gamma': 1, 'kernel': 'linear'}
Оценка:
               precision    recall  f1-score   support

       False    0.86632   0.95828   0.90998      9109
        True    0.75578   0.46611   0.57661      2523

    accuracy                        0.85153     11632
   macro avg    0.81105   0.71220   0.74330     11632
weighted avg    0.84234   0.85153   0.83767     11632

